# 1. 패키지 설치

In [1]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Knowledge Base 구성을 위한 데이터 생성

- Chroma를 활용한 [3.2 LangChain과 Chroma를 활용한 RAG 구성](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.2%20LangChain%EA%B3%BC%20Chroma%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20RAG%20%EA%B5%AC%EC%84%B1.ipynb)과 동일함
- Vector Database만 [Pinecone](https://www.pinecone.io/)으로 변경

In [12]:
from dotenv import load_dotenv

In [13]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./Guide.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [7]:
document_list[1]

Document(metadata={'source': './Guide.docx'}, page_content="Page 10: 8. Glossary of Key Terms\n\n8.1 Common Financial Terminology\n\n8.2 Document Control & Revisions\n\n\n\n1. Introduction to Financial Analysis\n\n1.1 Purpose of the Guide\n\nWelcome to the Financial Analyst's Guide. This document is designed to be a quick reference for all analysts at Unity Financial Group. Its purpose is to standardize our analytical approach, provide a quick overview of key metrics, and serve as a reliable source for internal reporting standards. By following this guide, you ensure the accuracy and consistency of all financial analysis within our organization.\n\n1.2 Our Core Analytical Principles\n\nOur analysis is based on three core principles: Accuracy, Clarity, and Timeliness.\n\nAccuracy: All data must be sourced from approved databases and verified against official company reports.\n\nClarity: Analytical findings should be presented in a clear, concise manner, free of unnecessary jargon.\n\nTi

In [14]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [15]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'guide-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
# query = 'What does it mean if the current ratio is 1.75?'

# 3. 답변 생성을 위한 Retrieval

- `RetrievalQA`에 전달하기 위해 `retriever` 생성 / 쉽게 말하면, **"답변에 참고할 자료를 찾아주는 검색기"**라고 이해하면 됩니다.
- `search_kwargs` 의 `k` 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음 /너무 적게 잡으면 정보가 부족할 수 있고, 너무 많으면 불필요한 내용이 포함될 수 있습니다.
- `.invoke()` 를 호출해서 어떤 문서를 가져오는지 확인 가능 /쉽게 말하면 "실제로 검색해보고 결과를 확인하는 버튼" 같은 역할입니다.

In [ ]:
# retriever = database.as_retriever(search_kwargs={'k': 4}) # 검색할 때 최대 4개의 문서를 가져오도록 지정합니다.
# retriever.invoke(query)

[Document(id='a732bfa3-7235-415f-8026-a191ec3e872f', metadata={'source': './Guide.docx'}, page_content="Quick Ratio (Acid-Test Ratio): Similar to the current ratio but excludes inventory.\n\nFormula: (Current Assets - Inventories) / Current Liabilities\n\nInterpretation: A ratio above 1.0 suggests a strong ability to meet immediate liabilities.\n\n2.2 Profitability Ratios\n\nNet Profit Margin: Indicates how much net income is generated as a percentage of revenue.\n\nFormula: Net Income / Revenue\n\nInterpretation: A higher percentage indicates better profitability.\n\nReturn on Equity (ROE): Measures the profitability in relation to shareholder equity.\n\nFormula: Net Income / Shareholder's Equity\n\nInterpretation: ROE indicates how effectively management is using shareholder funds to generate profit.\n\n\n\n3. Market Risk Analysis Framework\n\n3.1 Understanding Market Risk\n\nMarket risk is the possibility of losses in an investment due to factors that affect the overall performance 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [ ]:
# from langchain import hub

# prompt = hub.pull("rlm/rag-prompt") # 쉽게 말하면 **LLM에 전달할 “질문-답변 지침서”**를 가져온다고 생각하면 됩니다.

In [ ]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model='gpt-4o') # 이 객체는 실제로 질문을 받아 답변을 생성하는 역할을 합니다.

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [ ]:
# from langchain.chains import RetrievalQA


# qa_chain = RetrievalQA.from_chain_type(
#     llm, 
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt}
# )

# RetrievalQA.from_chain_type(...) 의미
# llm : 질문을 답변으로 변환할 모델 지정
# retriever : 질문과 관련된 문서를 검색하는 역할 지정
# chain_type_kwargs={"prompt": prompt} : LLM에 전달할 프롬프트 템플릿 지정

# 동작 과정:
# 사용자가 질문(query)을 입력하면
# retriever가 관련 문서를 검색 → 최대 4개(k=4)
# 검색된 문서와 질문을 llm + prompt와 함께 전달
# LLM이 최종 답변 생성

# 즉, 이 코드는 문서 검색 + LLM 답변 생성을 하나의 체인으로 묶은 것입니다. ==> 질문 → 관련 문서 검색(retriever) → 프롬프트와 함께 LLM에게 전달 → 답변 생성


In [ ]:
# LangChain 권장사항에 따라 강의 코드와 다르게 `.invoke()`를 사용합니다
# ai_message = qa_chain.invoke({"query": query})


# 1. qa_chain.invoke(...)
# qa_chain는 이전에 생성한 RetrievalQA 체인입니다.
# .invoke()는 이 체인을 실제로 실행시키는 함수입니다.
# 즉, 체인 안의 검색(retriever) → LLM 답변 생성 과정을 한 번에 수행합니다.

# 사용자 질문(query) → retriever로 관련 문서 검색 → LLM이 답변 생성 → ai_message에 저장

In [ ]:
# ai_message

{'query': 'What does it mean if the current ratio is 1.5?',
 'result': 'A current ratio of 1.5 means that the company has $1.50 in current assets for every $1.00 in current liabilities. This suggests that the company has a reasonable ability to pay off its short-term obligations, although it is below the ideal ratio of 2.0, which is generally considered healthy.'}